In [ ]:
# Clone the nvidia TensorRT LLM repository

!git clone https://github.com/NVIDIA/TensorRT-LLM.git
%cd TensorRT-LLM/examples/llama

Cloning into 'TensorRT-LLM'...
remote: Enumerating objects: 9404, done.
remote: Counting objects: 100% (3156/3156), done.
remote: Compressing objects: 100% (1142/1142), done.
remote: Total 9404 (delta 2199), reused 2704 (delta 1997), pack-reused 6248
Receiving objects: 100% (9404/9404), 126.63 MiB | 41.01 MiB/s, done.
Resolving deltas: 100% (6471/6471), done.
Updating files: 100% (1928/1928), done.
/content/TensorRT-LLM/examples/llama


In [ ]:
# Install necessary python dependencies

!pip install tensorrt_llm -U --pre --extra-index-url https://pypi.nvidia.com
!pip install huggingface_hub pynvml mpi4py
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.9/327.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Download the Mistral 7B Instruct v0.2 weights from hugging face

from huggingface_hub import snapshot_download
from google.colab import userdata


snapshot_download(
    "mistralai/Mistral-7B-Instruct-v0.2",
    local_dir="tmp/hf_models/mistral-7b-instruct-v0.2",
    max_workers=4
)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

'/content/tmp/hf_models/mistral-7b-instruct-v0.2'

In [ ]:
# Convert the raw model weights into tensorrt-llm checkpoint format

!python convert_checkpoint.py --model_dir ./tmp/hf_models/mistral-7b-instruct-v0.2 \
                             --output_dir ./tmp/trt_engines/1-gpu/ \
                             --dtype float16

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
[TensorRT-LLM] TensorRT-LLM version: 0.9.0.dev20240206000.9.0.dev2024020600
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.52s/it]
Weights loaded. Total time: 00:00:00
Total time of converting checkpoints: 00:01:14


In [ ]:
# Compile the model

!trtllm-build --checkpoint_dir ./tmp/trt_engines/1-gpu/ \
            --output_dir ./tmp/trt_engines/compiled-model/ \
            --gpt_attention_plugin float16 \
            --gemm_plugin float16 \
            --max_input_len 32256


[TensorRT-LLM] TensorRT-LLM version: 0.9.0.dev2024020600[02/19/2024-00:33:46] [TRT-LLM] [I] Set bert_attention_plugin to float16.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set gpt_attention_plugin to float16.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set gemm_plugin to float16.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set lookup_plugin to None.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set lora_plugin to None.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set context_fmha to True.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set context_fmha_fp32_acc to False.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set paged_kv_cache to True.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set remove_input_padding to True.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set use_custom_all_reduce to True.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set multi_block_mode to False.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set enable_xqa to True.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set attention_qk_half_accumulation to False.
[02/19/2024-00:33:46] [TRT-LLM] [I] Set tokens_per_block to 12

In [ ]:
# Upload the compiled model to hugging face hub

import os
from huggingface_hub import HfApi

for root, dirs, files in os.walk(f"tmp/trt_engines/compiled-model", topdown=False):
    for name in files:
        filepath = os.path.join(root, name)
        filename = "/".join(filepath.split("/")[-2:])
        print("uploading file: ", filename)
        api = HfApi(token=userdata.get('HF_WRITE_TOKEN'))
        api.upload_file(
            path_or_fileobj=filepath,
            path_in_repo=filename,
            repo_id="htrivedi99/mistral-7b-v0.2-trtllm"
        )

uploading file:  compiled-model/rank0.engine


rank0.engine:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

uploading file:  compiled-model/config.json


In [ ]:
%cd ..
!pwd

/content/TensorRT-LLM/examples
/content/TensorRT-LLM/examples


In [ ]:
# Test the compiled model

!python3 run.py --max_output_len=256 \
               --tokenizer_dir ./llama/tmp/hf_models/mistral-7b-instruct-v0.2/ \
               --engine_dir=./llama/tmp/trt_engines/compiled-model \
               --max_attention_window_size=4096

[TensorRT-LLM][INFO] Engine version 0.9.0.dev2024020600 found in the config file, assuming engine(s) built by new builder API.
[TensorRT-LLM][WARNING] Parameter max_draft_len cannot be read from json:
[TensorRT-LLM][WARNING] [json.exception.out_of_range.403] key 'max_draft_len' not found
[TensorRT-LLM][WARNING] [json.exception.type_error.302] type must be string, but is null
[TensorRT-LLM][WARNING] Optional value for parameter quant_algo will not be set.
[TensorRT-LLM][WARNING] [json.exception.type_error.302] type must be string, but is null
[TensorRT-LLM][WARNING] Optional value for parameter kv_cache_quant_algo will not be set.
[TensorRT-LLM][INFO] MPI size: 1, rank: 0
[TensorRT-LLM][INFO] Loaded engine size: 13815 MiB
[TensorRT-LLM][INFO] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 13976, GPU 14257 (MiB)
[TensorRT-LLM][INFO] [MemUsageChange] Init cuDNN: CPU +1, GPU +10, now: CPU 13977, GPU 14267 (MiB)
[TensorRT-LLM][INFO] [MemUsageChange] TensorRT-managed allocat